In [5]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import datetime as d

In [2]:
class Website:
  '''
  다음 뉴스 사이트의 구조에 관한 정보를 저장할 클래스

  start_date: 크롤링 대상 기간의 시작일, 8자리 숫자. (예시) 20210101
  end_date: 크롤링 대상 기간의 종료일, 8자리 숫자. (예시) 20210402
  start_page: 크롤링을 시작할 뉴스 사이트의 페이지 번호. (예시) 500
  end_page: 크롤링을 종료할 뉴스 사이트의 페이지 번호. (예시) 502

  사이트가 수정된 경우, __init__ 안의 항목을 수정합니다.
  '''

  def __init__(self, start_date, end_date):
    self.link = 'a', 'class', 'link_txt'  # 메인 페이지(https://news.daum.net/breakingnews/?page=1&regDate=20210403)에서 개별 뉴스 링크(https://entertain.v.daum.net/v/20210403104425168)로
    self.link_xpath = '/html/body/div[2]/div[3]/div/div[1]/div[2]/ul[2]/li[', ']/div[2]/strong/a' # 메인 페이지(https://news.daum.net/breakingnews/?page=1&regDate=20210403)에서 개별 뉴스 링크(https://entertain.v.daum.net/v/20210403104425168)로 full xpath
    self.num_news = 50  # 메인 페이지 내 기사 갯수
    self.news_date = 'num_date'  # 개별 뉴스 링크 안의 기사 작성일자 class
    self.more = 'link_fold'  # 더보기 버튼 class

    start_date = str(start_date)
    end_date = str(end_date)

    self.file_name = 'daum_news_comments_' + start_date + '~' + end_date + '.csv'  # 크롤링 중 데이터를 저장할 파일명

    start_date = d.date(int(start_date[:4]), int(start_date[4:6]), int(start_date[6:8]))
    end_date = d.date(int(end_date[:4]), int(end_date[4:6]), int(end_date[6:8]))
    
    term = (end_date - start_date).days
    
    self.date_list = []
    for i in range(term + 1):
      day = (start_date + d.timedelta(days=i)).strftime('%Y%m%d')
      self.date_list.append(day)

  def urls(self):
    res = []
    for date in self.date_list:
      res.append('https://news.daum.net/ranking/bestreply/?regDate={}'.format(date))
    return res

In [3]:
# colab에서 selenium 사용하는 방법
# https://gist.github.com/korakot/5c8e21a5af63966d80a676af0ce15067
# 자바스크립트 스크레이핑을 위해서 필요
!apt update
!apt install chromium-chromedriver
!pip install selenium

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [52.7 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:13 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:14 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packag

In [4]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import csv

In [6]:
class Crawler(Website):

  def getPage(self, url):
    try:
      req = requests.get(url)
    except requests.exceptions.RequestException:
      return None
    return BeautifulSoup(req.text, 'html.parser')

  def comments(self, news_date, wd, w):
    try:      
      # comments list
      comments = wd.find_elements_by_class_name('list_comment')[0].text.split('\n')
      # comments list 분석
      parse_comments = []
      temp = []
      parse = 0
      for comment in comments:
        temp.append(comment)
        if comment == '댓글 찬성하기':
          parse = 1
        elif parse == 1 and comment == '댓글 비추천하기':
          parse = 2
        elif parse == 2:
          parse_comments.append(temp)
          temp = []
          parse = 0
      count = 0
      for comment in parse_comments:
        # 댓글 작성일자
        comment_date = comment[1]
        if comment_date[-1] == '전':
          comment_date = d.datetime.today().strftime('%Y%m%d')
        else:
          comment_date = comment_date.split('. ')
          comment_date = ''.join(comment_date[:3])
        # 댓글 내용
        comment_content = ' '.join(comment[2:-5])
        # 좋아요, 싫어요
        likes = comment[-3]
        dislikes = comment[-1]
        # csv 저장
        w.writerow((news_date, comment_date, comment_content, likes, dislikes))
        print(news_date, comment_date, comment_content, likes, dislikes)
        count += 1
      print('<found', count, 'comments>')
    except:
      pass    

  def parse(self):
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    wd = webdriver.Chrome(options=options)
    with open('/content/drive/MyDrive/project_data/' + self.file_name, 'w+') as f:
      w = csv.writer(f)
      # csv 파일 Column 이름
      w.writerow(('기사 작성일', '댓글 작성일', '댓글 내용', '좋아요', '싫어요'))
      for url in self.urls():
        wd.get(url)
        # 기사 링크 리스트 작성
        bs = self.getPage(url)
        print(url)
        links = bs.findAll(self.link[0], {self.link[1] : self.link[2]})[:self.num_news]
        # 각 개별 기사 링크로 이동
        for link in links:
          link = link.attrs['href'], link.text
          wd.get(link[0])
          print(link[0], link[1])
          time.sleep(3)
          try:
            # 기사 작성일자
            news_date = wd.find_elements_by_class_name(self.news_date)[0].text
            news_date = news_date.split('. ')
            news_date = ''.join(news_date[:3])
            # 댓글 '더보기' 버튼 클릭
            while wd.find_elements_by_class_name(self.more)[0].text == '더보기':
              wd.find_elements_by_class_name(self.more)[0].click()
              time.sleep(0.5)
          except:
            print('error')
          # 댓글 저장
          self.comments(news_date, wd, w)
    wd.close()

In [7]:
%%time

crawler = Crawler(20210101, 20210131)
crawler.parse()

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
20210126 20210126 짐당이 그렇게 높은 여론지지율이라니 그게 더 놀랍다 799 43
20210126 20210126 국민들이 멍청하지 않다.  총선에서 180석 가까이 나온 의미는  적어도 다음 정권 까진   민주당에게 투표하겠다는 국민이 다수 였다는 것. 2715 253
20210126 20210126 국짐당에는 다 쓰레기만 모였나봐 781 68
20210126 20210126 그 지지율이 실존했는지 의문이야~ 언론이 만들어낸건 아닌지~ 1114 29
20210126 20210126 유승민 오세훈 나경원 홍준표 원희룡 이것들이 대권후보???푸하하하 민주당에선 땡큐라고 할거다 진짜 인물 없다 한심하다 이것들 몇년뒤엔 김문수 조원진 따라 광화문 태극기 따라 다닐걸 한심하다 한심해 4309 92
20210126 20210126 정치에 발을 들여밀지않은 윤석열씨나 허구헌날 말아톤이나 뛴 안씨가 도찐개찐. ㅋㅋㅋㅋㅋ 욱긴다. ㅋㅋㅋㅋㅋ 731 54
20210126 20210126 국민들이 멍청한줄 알지? 태극기부대  수구꼴통들 제외하고 국민들은 똑똑하단다 3332 95
20210126 20210126 내 생각에는 윤석렬 지지율 조작해서 정권 흔들다가,, 국민의 짐당에 돌덩이 될거 같으니,, 서서히 정리 작업 들어 가는 거 같다. 1400 74
20210126 20210126 국민의 암당은 그냥 소멸하고 있는 당입니다  시대 정신에 맞지 않기 때문입니다  마지막 발악이 추접하고 위험해서 그게 걱정입니다 1640 69
<found 15 comments>
https://news.v.daum.net/v/20210126100102474 文 캠프 출신 전인범 장군 "한미훈련 못하면 미군 떠날 것"
20210126 20210126 또 똥아 북풍 추억 떠올리고  소설을 쓴다. 폐간. 287 86
20210126 20210126 미군떠난다는 말은 한 70년했나? 352 30
20210126 20210126 